#### This Notebook is based on my learnings from the below sources 
##### 1. https://til.simonwillison.net/llms/python-react-pattern
##### 2. AI Agents in LangGraph, from deeplearning.ai

##### In this notebook we will code a ReAct Agent from scratch.

In [52]:
import openai
from openai import OpenAI
from dotenv import load_dotenv
import os

In [53]:
# Load environment variables from .env file.
load_dotenv()
# load your openai api key from .env file
openai_key = os.getenv("API_KEY")
client = OpenAI(api_key=openai_key)

In [54]:
# lets crearte an Agent class with system prompt as a parameter, which will allow the user to input the system prompt as needed.

In [56]:
class Agent:
    def __init__(self, system=""):
        self.system = system
        self.messages = [] # messages are saved in this list to keep track of messages over time.
        if self.system:
            self.messages.append({"role": "system", "content": system}) # system message is the instructions that we provide to llm.

    def __call__(self, message):
        self.messages.append({"role": "user", "content": message}) # user message is what the user will the agent to do.
        result = self.execute()
        self.messages.append({"role": "assistant", "content": result}) # assistant message is what the llm returns for the user query.
        return result

    def execute(self):
        completion = client.chat.completions.create(
                        model="gpt-4o", 
                        temperature=0,
                        messages=self.messages)
        return completion.choices[0].message.content
    

In [57]:
# ReAct prompt that we give to the llm as system message. 

system_prompt = '''You operate in a loop of Thought, Action, PAUSE, Observation.  
At the end of the loop, you output an Answer.  

Use Thought to describe your reasoning about the question you've been asked.  
Use Action to execute one of the available actions and then return PAUSE.  
Observation will be the result of running that action.  

While answering First try to look at the given tools only.

Your available actions are:  

1. **calculate**:  
   e.g., calculate: (5 + 3) * 2  
   Executes a mathematical operation and returns the result.  

2. **get_employee_salary**:  
   e.g., get_employee_salary: 30  
   Takes the employee name and return the salary.  

3. **get_employee_name**:  
   e.g., get_employee_name: karthik  
   Takes the id of the employee and provide the Name.  

### Example Session:  

Question: What is the salary of Ritu?  
Thought: I need to look up the Salary of Ritu.  
Action: get_employee_salary: Ritu  
PAUSE

When called again with:  

Observation: The salary of Ritu is 20.  

You then output:  
Answer: The salary of Ritu is 20.  

Question: What is 12 * 8 + 5?  
Thought: I should perform this calculation using the calculate tool.  
Action: calculate: 12 * 8 + 5  
PAUSE 

You will be called again with:  

Observation: The result is 101.  

You then output:  
Answer: The result of 12 * 8 + 5 is 101.  

'''.strip()

In [81]:
## These are the tools/Functions that the llm use to get the answer to user query.

def calculate(what):
    return eval(what)

def get_employee_salary(employee_name):
    '''Takes the employee name and return the salary'''
    
    if 'rahul' in employee_name.lower():
        return '40'
    else:
        return '20'
        
def get_employee_name(employee_id):
    '''Takes the id of the employee and provide the Name'''
    
    if 'ad241' in employee_id.lower():
        return 'Rahul'
    else:
        return 'other'
    
known_actions = {"calculate":calculate,
                 "get_employee_salary":get_employee_salary,
                 "get_employee_name":get_employee_name
                }

In [42]:
# user query
user_prompt = 'What is the salary of Ratan'
abot = Agent(system_prompt)
result = abot(user_prompt)
print(result)

# Notice how the llm output only till 'Action' but not the final answer. 
# This is because in the system prompt we asked it to pause after identifying the action.
# After that we manually provide the Action to next function.


Thought: I need to look up the salary of Ratan.  
Action: get_employee_salary: Ratan  
PAUSE


In [43]:
# In the above output the llm is asking to pass 'Ratan' in 'get_employee_salary' tool(function). Lets do that.

In [44]:
result = get_employee_salary("Ratan")
result
# Now with this result we pass this to the llm again to get the final answer.

'20'

In [45]:
next_prompt = "Observation: {}".format(result)
abot(next_prompt)

'Answer: The salary of Ratan is 20.'

In [27]:
# You can see the list of messages here and how llm responds at each step
abot.messages

[{'role': 'system',
  'content': "You operate in a loop of Thought, Action, PAUSE, Observation.  \nAt the end of the loop, you output an Answer.  \n\nUse Thought to describe your reasoning about the question you've been asked.  \nUse Action to execute one of the available actions and then return PAUSE.  \nObservation will be the result of running that action.  \n\nWhile answering First try to look at the given tools only.\n\nYour available actions are:  \n\n1. **calculate**:  \n   e.g., calculate: (5 + 3) * 2  \n   Executes a mathematical operation and returns the result.  \n\n2. **get_employee_salary**:  \n   e.g., get_employee_salary: 30  \n   Takes the employee name and return the salary.  \n\n3. **get_employee_name**:  \n   e.g., get_employee_name: karthik  \n   Takes the id of the employee and provide the Name.  \n\n### Example Session:  \n\nQuestion: What is the salary of Ritu?  \nThought: I need to look up the Salary of Ritu.  \nAction: get_employee_salary: Ritu  \nPAUSE\n\nWhen

In [46]:
### new user prompt
user_prompt = 'What is the average salary of Rahul and Saurabh?'
abot = Agent(system_prompt)
result = abot(user_prompt)
print(result)

result1 = get_employee_salary('Rahul')
next_prompt1 = "Observation: {}".format(result1)
result2 = abot(next_prompt1)
print(result2)

result2 = get_employee_salary('Saurabh')
next_prompt2 = "Observation: {}".format(result2)
result3 = abot(next_prompt2)
print(result3)

Thought: I need to find the salaries of Rahul and Saurabh first, and then calculate their average.  
Action: get_employee_salary: Rahul  
PAUSE
Thought: I have found the salary of Rahul, which is 40. Now, I need to find the salary of Saurabh.  
Action: get_employee_salary: Saurabh  
PAUSE
Thought: I have found the salaries of Rahul and Saurabh. Rahul's salary is 40, and Saurabh's salary is 20. Now, I need to calculate their average salary.  
Action: calculate: (40 + 20) / 2  
PAUSE


In [47]:
result4 = calculate('(40 + 20) / 2')
next_prompt4 = "Observation: {}".format(result4)
result5 = abot(next_prompt4)
print(result5)

Answer: The average salary of Rahul and Saurabh is 30.0.


### Autmate the process.

In [84]:
## lets automate the above steps in a loop, without manual intervention.
## The function below is nothing but an AI Agent.

In [82]:
import re
action_re = re.compile(r'^Action: (\w+): (.*)$')

def query_now(question,max_loop=6):
    i=0
    abot = Agent(system_prompt)
    next_prompt = question
    
    while i < max_loop:
        i += 1
        result = abot(next_prompt)
        print(result)
        actions = [
            action_re.match(a) 
            for a in result.split('\n') 
            if action_re.match(a)
        ]
        print(actions)
        if actions:
            # There is an action to run
            action, action_input = actions[0].groups()
            if action not in known_actions:
                raise Exception("Unknown action: {}: {}".format(action, action_input))
            print(" -- running {} {}".format(action, action_input))
            observation = known_actions[action](action_input)
            print("Observation:", observation)
            next_prompt = "Observation: {}".format(observation)
        else:
            return

In [83]:
question = 'What is the Average salary of Karthik and employee id AD241?'

query_now(question)


Thought: I need to find the salary of Karthik and the employee with ID AD241, then calculate their average salary. First, I'll get the salary of Karthik.  
Action: get_employee_salary: Karthik  
PAUSE
[<re.Match object; span=(0, 38), match='Action: get_employee_salary: Karthik  '>]
 -- running get_employee_salary Karthik  
Observation: 20
Thought: The salary of Karthik is 20. Now, I need to find the name of the employee with ID AD241 to get their salary.  
Action: get_employee_name: AD241  
PAUSE
[<re.Match object; span=(0, 34), match='Action: get_employee_name: AD241  '>]
 -- running get_employee_name AD241  
Observation: Rahul
Thought: The employee with ID AD241 is Rahul. Now, I need to find the salary of Rahul.  
Action: get_employee_salary: Rahul  
PAUSE
[<re.Match object; span=(0, 36), match='Action: get_employee_salary: Rahul  '>]
 -- running get_employee_salary Rahul  
Observation: 40
Thought: The salary of Rahul is 40. Now, I will calculate the average salary of Karthik and Rah